In [1]:
from datetime import datetime
import os
import pickle
import neat
import numpy as np
from Neat.evaluation_util import DataManager
from Neat.selection_util import TournamentReproduction, FitnessPropReproduction
from data_load_util import load_state_data, make_dataset
from tqdm import tqdm
import random
from Neat.saving_util import *
from Neat.neat_util import *

Loading data_manager for NEAT
Loading Data
number of zip codes: 10559
number of zip codes with solar data: 10559
number of zip codes with census data: 10559
Removing Outliers
zips after removing outliers: 7795


In [2]:
config_path = 'Neat/neat-config'

#load datasets
combined_df = make_dataset(remove_outliers=True)
state_df = load_state_data(combined_df, load="Clean_Data/data_by_state.csv")
data_manager = DataManager(combined_df, state_df)
k_folds = 5

Loading Data
number of zip codes: 10559
number of zip codes with solar data: 10559
number of zip codes with census data: 10559
Removing Outliers
zips after removing outliers: 7795


In [ ]:
#single model run
network = run(config_path, eval_genomes_lexicase)
save_model(network, model_name="NEAT_model_lexicase.pkl")
cv_order = run_network(network, data_manager, cross_val=True)

loading configuration...
creating population...
setting reporters...
training model...

 ****** Running generation 0 ****** 



100%|██████████| 100/100 [05:37<00:00,  3.38s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 5.09091 - size: (1, 6) - species 1 - id 22
Average adjusted fitness: nan
Mean genetic distance 1.701, standard deviation 0.522
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100      5.1      nan     0
Total extinctions: 0
Generation time: 411.836 sec

 ****** Running generation 1 ****** 



100%|██████████| 100/100 [06:21<00:00,  3.82s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 5.26051 - size: (1, 6) - species 1 - id 191
Average adjusted fitness: nan
Mean genetic distance 1.775, standard deviation 0.435
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100      5.3      nan     0
Total extinctions: 0
Generation time: 477.933 sec (444.885 average)

 ****** Running generation 2 ****** 



100%|██████████| 100/100 [05:55<00:00,  3.56s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 5.15675 - size: (1, 5) - species 1 - id 296
Average adjusted fitness: nan
Mean genetic distance 1.942, standard deviation 0.537
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    2    63      5.2      nan     1
     2    0    37       --       --     0
Total extinctions: 0
Generation time: 427.770 sec (439.180 average)
Saving checkpoint to Neat/checkpoints/neat-checkpoint-2

Best individual in generation 2 meets fitness threshold - complexity: (1, 6)
directory name: Neat/models\12-18-24
making directory


IndexError: list index out of range

In [8]:
cv_order = run_network(network, data_manager, cross_val=False)

NUM_PANELS = 10000
print("Carbon Offset score: ",data_manager.score(cv_order, 'carbon_offset', NUM_PANELS, train=True))

print("Energy Generation score: ",data_manager.score(cv_order, 'energy_generation', NUM_PANELS, train=True))

print("Income Equity score: ",data_manager.score(cv_order, 'income_equity', NUM_PANELS, train=True))

print("Racial Equity score: ",data_manager.score(cv_order, 'racial_equity', NUM_PANELS, train=True))

Carbon Offset score:  0.8594651801604856
Energy Generation score:  0.8589490929704989
Income Equity score:  0.7258815625972799
Racial Equity score:  0.7824152665785489


In [ ]:

#run lexicase
lexi_network, lexi_results = K_fold_run(config_path, data_manager, eval_genomes_lexicase, k=k_folds)
save_model(lexi_network, lexi_results, model_name="NEAT_model_lexicase.pkl", results_name="lexicase_results.pkl")

#run fitness prop
fp_network, fp_results = K_fold_run(config_path, data_manager, eval_genomes_weighted_sum, reproduction_method=FitnessPropReproduction, k=k_folds)
save_model(fp_network, fp_results, model_name="NEAT_model_fitness_prop.pkl", results_name="fitness_prop_results.pkl")

#tournament selection
tourney_network, tourney_results = K_fold_run(config_path, data_manager, eval_genomes_weighted_sum, reproduction_method=TournamentReproduction, k=k_folds)
save_model(tourney_network, model_name="NEAT_model_tournament.pkl", results_name="tournament_results.pkl")

#run random selection
rand_network, rand_results = K_fold_run(config_path, data_manager, eval_genomes_random, k=k_folds)
save_model(rand_network, model_name="NEAT_model_random.pkl")

#print results
result_metrics = ['income_equity', 'racial_equity', 'carbon_offset', 'energy_generation']
for i, res in enumerate(lexi_results):
    print(f"Lexicase {result_metrics[i]}", res)

for i, res in enumerate(fp_results):
    print(f"Fitness prop {result_metrics[i]}", res)
    
for i, res in enumerate(tourney_results):
    print(f"Tourney {result_metrics[i]}", res)

for i, res in enumerate(rand_results):
    print(f"Random {result_metrics[i]}", res)

# for i, res in enumerate(rand_results):
#     print(f"Random {result_metrics[i]}", res)


NameError: name 'K_fold_run' is not defined